In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import math

with open('../input/hashcode-drone-delivery/busy_day.in') as file:
    line_list = file.read().splitlines()



In [ ]:
y_rows = line_list[0].split()[0]
x_columns = line_list[0].split()[1]
num_drones = line_list[0].split()[2]
max_turn = line_list[0].split()[3]
max_load = line_list[0].split()[4]


In [ ]:
weights = line_list[2].split()
    


In [ ]:
coordinates=[]
for i in range(4,4+10*2,2):
    coordinates.append(line_list[i])
    
coordinates = [x.split() for x in coordinates]



In [ ]:
item = [x for x in range(0,400)]
products = pd.DataFrame({'item': item, 'weights' : weights})

#item_weights = products.transpose()

products


In [ ]:
co_index = 0
warehouse_names = []
for i in range(5,25,2):
    products['warehouse_' + str(co_index)] = line_list[i].split()
    warehouse_names.append('warehouse_' + str(co_index))
    co_index+=1
    
    



    
    

In [ ]:
item_weights=products[['item','weights']]
item_weights

In [ ]:
warehouses = pd.DataFrame(coordinates, columns=['y_coordinate', 'x_coordinate'])
warehouses= warehouses.apply(pd.to_numeric)
warehouses['warehouse'] = warehouse_names
warehouses

In [ ]:
products = products.apply(pd.to_numeric)
products.dtypes

# Weights of Each Product

In [ ]:
item_weights

In [ ]:
order = line_list[25:]
delivery_cor = [order[i] for i in range(0, len(order),3)]
item_num = [order[i] for i in range(2, len(order), 3)]



In [ ]:
test_1 = [x.split() for x in item_num]
delivery_cor = [x.split() for x in delivery_cor]



In [ ]:
test_list = [[]]
order_id = 0
count = 0
for element in test_1:
    for instance in element:
        insert = [order_id, instance, delivery_cor[order_id][0], delivery_cor[order_id][1]]
        test_list.append(insert)
        count+=1
        
    order_id+=1
    
test_list.pop(0)
        

        

# Orders Made (with Destination Address)

In [ ]:
orders = pd.DataFrame(test_list, columns=['order_id','item','y_coordinate', 'x_coordinate'])
orders = orders.apply(pd.to_numeric) 
orders_merged  = orders.join(item_weights, on='item', rsuffix='_right').drop('item_right', axis=1)
orders_merged = orders_merged.apply(pd.to_numeric) 

In [ ]:
orders_merged

In [ ]:
tot_weight_for_orders = orders_merged[['order_id', 'weights','y_coordinate', 'x_coordinate']].groupby(['order_id','y_coordinate', 'x_coordinate']).sum()
tot_weight_for_orders = tot_weight_for_orders.reset_index()
tot_weight_for_orders

In [ ]:
higher_than_limit = tot_weight_for_orders[tot_weight_for_orders.weights>200][['y_coordinate', 'x_coordinate']]
lower_than_limit  = tot_weight_for_orders[tot_weight_for_orders.index.isin(higher_than_limit.index)==False][['y_coordinate', 'x_coordinate']]


# Map of Orders and Warehouses

In [ ]:
arr = np.array(coordinates, dtype=np.int32)

x=arr[:,1]
y=arr[:,0]




plt.figure(figsize=(21,14))
plt.scatter(x,y,s=160, c='green') #warehouses

for i, wh in enumerate(warehouse_names):
    plt.annotate(wh, (x[i], y[i]))

x= higher_than_limit.x_coordinate
y= higher_than_limit.y_coordinate

plt.scatter(x,y,s=20, c='red', marker='x') # order destinations with higher than limit

x= lower_than_limit.x_coordinate
y= lower_than_limit.y_coordinate

plt.scatter(x,y,s=20, c='blue', marker='x')# order destinations with lower than limit

plt.xlim(0, 700)
plt.ylim(0, 450)
plt.show()

# Warehouse Assignment for Orders

In [ ]:
print(products)
print('\n')
print('----')
print('\n')
print(warehouses)
print('\n')
print('----')
print('\n')
print(orders_merged)

In [ ]:
w_copy

In [ ]:
w_copy = warehouses.copy()
o_copy = orders_merged.copy()
p_copy = products.copy()

def sorted_distance(order_id):
    
    '''
    Sorts the warehouse according to the distance to order destination
    
    '''
    x = orders_merged[orders_merged['order_id']==order_id].x_coordinate.drop_duplicates().values[0]
    y = orders_merged[orders_merged['order_id']==order_id].y_coordinate.drop_duplicates().values[0]
    w_copy = warehouses.copy()
    w_copy['distance'] = (abs(x**2 - w_copy.x_coordinate**2) + abs(y**2 - w_copy.y_coordinate**2)).pow(1./2)
    w_copy['distance'] = w_copy['distance'].apply(lambda x: math.ceil(x))
    w_copy = w_copy.astype({'distance': 'int64'}).sort_values(by=['distance'])
    return w_copy.warehouse
    
    
    
    

In [ ]:

def check_availability(order_id, item):
    wh_to_check = sorted_distance(order_id)# which
    for wh in wh_to_check:
        if(p_copy[p_copy['item']==item][[wh]].iloc[0][0] > 0):
            p_copy.at[item, wh] -= 1    
            return wh
        
    print("No Availability") #Actually, this cell never runs as there is availability for all orders
            
    
    
    
    

In [ ]:
o_copy['From'] = o_copy.apply(lambda x: check_availability(x['order_id'], x['item']), axis=1) # assigning warehouse for each order
print("Done")

In [ ]:
o_copy[o_copy.item==11] #probably only warehouse_4 possesses item 11

In [ ]:
products[products.item==11] # yes, it does

In [ ]:
wh_orders = o_copy[['From','order_id']].groupby(['From']).count().reset_index()

x = wh_orders.iloc[:,0]
y = wh_orders.iloc[:,1]

plt.figure(figsize=(12,8))
plt.title("Assigned orders by warehouses\n")
sns.barplot(x=x, y=y)
plt.xlabel("\nWarehouse number")
plt.ylabel("Assigned orders")
plt.show()

As warehouse_1 is a remote warehouse, significantly lower number of orders assigned to this warehouse.

Next is sorting orders by (x, y) coordinates of destination. Because if load of drone does not exceed maximum load, a drone can carry extra item for nearby destination.

Drone should be better return to its origin warehouse because it is the shortest path it should take